In [ ]:
!pip install Flask

from collections import defaultdict
from flask import Flask, request, jsonify, Response
import pandas as pd
import numpy as np
import pickle
import json
import os

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install pyngrok
!ngrok authtoken 2Xiq3njrFD1bBTy6pCGqHjSxUaU_6V8fJioQysEkv6jmc7q9N
from pyngrok import ngrok
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=2bb752d5bd226c2ed03bff584e46b6a84b40fb6931f755fc34df0de478211935
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:

app = Flask(__name__)
run_with_ngrok(app)


In [ ]:
!pip install transformers
import pickle
from scipy.special import softmax
def preprocess(msg):
  X = []
  stop_words = set(nltk.corpus.stopwords.words("english"))
  tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
  tmp = []
  sentences = nltk.sent_tokenize(msg)
  for sent in sentences:
      sent = sent.lower()
      tokens = tokenizer.tokenize(sent)
      filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
      tmp.extend(filtered_words)
  X.append(tmp)

  with open('/content/drive/MyDrive/Ml Pickle/tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)
  X = tokenizer.texts_to_sequences(X)

  maxlen = 700
  X = pad_sequences(X, maxlen=maxlen)
  return X

def sent_result(msg):
  senti_model = "/content/drive/MyDrive/Ml Pickle/sentiment_model.pkl"
  senti_token = "/content/drive/MyDrive/Ml Pickle/tokenizer.pkl"

  with open(senti_model, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
  with open(senti_token, 'rb') as model_file:
    tokenizer = pickle.load(model_file)

  encoded_text = tokenizer(msg, return_tensors = 'pt')
  output = loaded_model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  val = 0
  acc = 0
  for i in range(3):
    if scores[i] > acc:
      acc = scores[i]
      val = i
  acc *= 100
  return [val, acc]

def initiate(msg):
  model_file_path = "/content/drive/MyDrive/Ml Pickle/model_pickle.pkl"
  with open(model_file_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
  result = loaded_model.predict(preprocess(msg))
  fin = float(result)

  temp = sent_result(msg)
  ans = [fin, temp]
  return ans;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.4 MB/s eta 0:00:00


In [ ]:
@app.route('/upload', methods=['POST'])
def start():
  msg = request.json['sentence']
  result = initiate(msg)

  return jsonify(result)

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://91a7-34-90-135-179.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
